In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

1. Scrape the current top 100 songs and their respective artists, and put the information into a pandas dataframe.

In [3]:
url = 'https://www.popvortex.com/music/charts/top-100-songs.php'

In [4]:
response = requests.get(url)
response.status_code # 200 status code means OK!

200

In [5]:
soup = BeautifulSoup(response.content, "html.parser")

In [6]:
#soup

In [7]:
#print(soup.prettify())

In [8]:
#soup.select(".title-artist")

In [9]:
title = []
artist = []

num_iter = len(soup.select(".title-artist"))
tClist = soup.select(".title-artist")

# iterate through the result set and retrive all the data
for i in range(num_iter):
    title.append(tClist[i].find(class_="title").get_text())
    artist.append(tClist[i].find(class_="artist").get_text())

print(artist)
print(title)






['Jimmy Buffett', 'Jimmy Buffett', 'Oliver Anthony Music', 'Zach Bryan', 'Gary Wright', 'Doja Cat', 'Smash Mouth', 'Jimmy Buffett', 'Jimmy Buffett', 'Paul Russell', 'Luke Combs', 'Jimmy Buffett', 'Gary Wright', 'Miley Cyrus', 'Nicki Minaj', 'Taylor Swift', 'Jimmy Buffett', 'Jelly Roll', 'Jimmy Buffett', 'Dua Lipa', 'Morgan Wallen', 'Jason Aldean', 'Oliver Anthony Music', 'Chris Stapleton', 'Smash Mouth', 'Smash Mouth', 'Lainey Wilson', 'Oliver Anthony Music', 'Jelly Roll', 'Jimmy Buffett', 'Nickelback', 'Smash Mouth', 'Morgan Wallen', 'Teddy Swims', 'Miley Cyrus', 'Smash Mouth', 'Fleetwood Mac', 'Bailey Zimmerman', 'Miley Cyrus', 'Olivia Rodrigo', 'Jimmy Buffett', 'Trump Latinos', 'Oliver Anthony Music', 'Smash Mouth', 'Wayland', 'Rema & Selena Gomez', 'Jimmy Buffett', 'Alan Jackson', 'Luke Combs', 'Billie Eilish', 'Nickelback', 'Jimmy Buffett', 'Creedence Clearwater Revival', 'Crosby, Stills & Nash', 'Rod Stewart', 'Jimmy Buffett', 'Timbaland, Nelly Furtado & Justin Timberlake', 'Jimm

In [10]:
songs = pd.DataFrame({"artist":artist,
                       "title":title,
                      })

In [11]:
songs

,artist,title
0,Jimmy Buffett,Margaritaville
1,Jimmy Buffett,Come Monday
2,Oliver Anthony Music,Rich Men North of Richmond
3,Zach Bryan,I Remember Everything (feat. Kacey Musgraves)
4,Gary Wright,Dream Weaver
...,...,...
92,Shinedown,A Symptom Of Being Human
93,Chicago,If You Leave Me Now
94,"USHER, Summer Walker & 21 Savage",Good Good
95,Jimmy Buffett,One Particular Harbour


SEARCH ENGINE

In [12]:
import random

In [13]:
def search_random_title(input_title):
    if input_title in songs['title'].values:
        while True:
            random_title = random.choice(songs['title'])
            if random_title != input_title:
                return random_title
    else:
        return "Sorry, we don't have this title."

input_title = input("Enter a song title: ")
search_random_title(input_title)

"Sorry, we don't have this title."

Adding more songs

In [74]:
secrets_file = open("secret.txt","r")

In [75]:
string = secrets_file.read()
string.split('\n')
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        #print(line.split(':'))
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()

In [76]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                           client_secret=secrets_dict['clientsecret']))

In [77]:

playlist = sp.user_playlist_tracks("spotify", "54nv8jbrm4JoHEZ49Qvjgl")

In [78]:
playlist["total"] 

9925

In [79]:
len(playlist["items"])

100

In [80]:
playlist['next']

'https://api.spotify.com/v1/playlists/54nv8jbrm4JoHEZ49Qvjgl/tracks?offset=100&limit=100&additional_types=track'

In [81]:
from random import randint
from time import sleep

def get_playlist_tracks(playlist_id):
    results = sp.user_playlist_tracks("spotify",playlist_id)
    tracks = results['items']
    while results['next']!=None:
        results = sp.next(results)
        tracks = tracks + results['items']
        sleep(randint(1,3000)/1000) # respectful nap
    return tracks

In [82]:
all_tracks = get_playlist_tracks("54nv8jbrm4JoHEZ49Qvjgl")
len(all_tracks)

9925

In [29]:
from pandas import json_normalize

In [38]:
spotify = json_normalize(all_tracks)
#spotify

In [37]:
spotify.columns

Index(['added_at', 'is_local', 'primary_color',
       'added_by.external_urls.spotify', 'added_by.href', 'added_by.id',
       'added_by.type', 'added_by.uri', 'track.album.album_type',
       'track.album.artists', 'track.album.available_markets',
       'track.album.external_urls.spotify', 'track.album.href',
       'track.album.id', 'track.album.images', 'track.album.name',
       'track.album.release_date', 'track.album.release_date_precision',
       'track.album.total_tracks', 'track.album.type', 'track.album.uri',
       'track.artists', 'track.available_markets', 'track.disc_number',
       'track.duration_ms', 'track.episode', 'track.explicit',
       'track.external_ids.isrc', 'track.external_urls.spotify', 'track.href',
       'track.id', 'track.is_local', 'track.name', 'track.popularity',
       'track.preview_url', 'track.track', 'track.track_number', 'track.type',
       'track.uri', 'video_thumbnail.url', 'track'],
      dtype='object')

In [95]:
#spotify["track.artists"]

In [61]:
def list_to_dict(x):
    if isinstance(x, list):
        return {i: x[i] for i in range(len(x))}
    else:
        return {} 

In [94]:
spotify['artist_dict'] = spotify['track.artists'].apply(lambda x: list_to_dict(x))
#spotify

In [93]:
#spotify["track.artists"]

In [64]:
def expand_list_dict2(row):
    df = pd.DataFrame.from_dict(row['artist_dict'], orient='index')
    df['song_id'] = row['track.id']
    return df

In [65]:
spotify['artists_dfs'] = spotify.apply(expand_list_dict2, axis=1)
spotify['artists_dfs'][3]

,external_urls,href,id,name,type,uri,song_id
0,{'spotify': 'https://open.spotify.com/artist/3...,https://api.spotify.com/v1/artists/3AQRLZ9PuTA...,3AQRLZ9PuTAozP28Skbq8V,The Script,artist,spotify:artist:3AQRLZ9PuTAozP28Skbq8V,4Musyaro0NM5Awx8b5c627


In [92]:
artist_df2 = pd.DataFrame(columns=['external_urls', 'href', 'id', 'name', 'type', 'uri'])
for val in spotify['artists_dfs']:
    #display(val)
    artist_df2 = pd.concat([artist_df2, val], axis=0)
    
#artist_df2

In [91]:
df_merged2 = pd.merge(left=spotify,
                    right=artist_df2,
                    how='inner',
                    left_on='track.id',
                    right_on='song_id')
#df_merged2

In [90]:
spotify_songs = df_merged2[['track.name', 'name', 'song_id']]
#spotify_songs

Attributes

In [85]:
chunks = [(i, i+100) for i in range(0, len(spotify_songs), 100)]
audio_features_list = []
for chunk in chunks:
    id_list100 = spotify_songs['song_id'][chunk[0]:chunk[1]]
    audio_features_list = audio_features_list + sp.audio_features(id_list100)
    sleep(randint(1,3000)/1000)
len(audio_features_list)

12236

In [86]:
audio_features_df = json_normalize(audio_features_list)

In [89]:
df_w_audio_ft = pd.merge(left=spotify_songs,
                        right=audio_features_df,
                        how='inner',
                        left_on='song_id',
                        right_on='id')
#df_w_audio_ft

In [88]:
df_w_audio_ft.isna().sum()

track.name          0
name                0
song_id             0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
type                0
id                  0
uri                 0
track_href          0
analysis_url        0
duration_ms         0
time_signature      0
dtype: int64